In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
# Try XGBoost
try:
    from xgboost import XGBClassifier
    HAS_XGB = True
except:
    HAS_XGB = False
    print("[Info] XGBoost not installed.")

In [ ]:
# --- Setup ---
os.makedirs("outputs", exist_ok=True)

In [ ]:
def savefig(path):
    plt.savefig(path, dpi=150, bbox_inches='tight')
    plt.close()

In [ ]:
# --- Load cleaned data from Project 1 ---
df = pd.read_csv("outputs/Collected_Data.csv")

In [ ]:
# --- Manual labels (Tanvir’s rule) ---
# High Performer if accuracy ≥ 0.97 and reaction time ≤ 1.30
df["MP2_Label"] = ((df["Accuracy"] >= 0.97) & (df["Average Reaction Time"] <= 1.30)).astype(int)

In [ ]:
# --- Statistical features ---
features = pd.DataFrame()
features["mean"] = df[["Accuracy","Average Reaction Time"]].mean(axis=1)
features["std"] = df[["Accuracy","Average Reaction Time"]].std(axis=1)
features["min"] = df[["Accuracy","Average Reaction Time"]].min(axis=1)
features["max"] = df[["Accuracy","Average Reaction Time"]].max(axis=1)

In [ ]:
X = features.values
y = df["MP2_Label"].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
# --- Neural Network ---
nn = MLPClassifier(hidden_layer_sizes=(50,50), max_iter=500, random_state=42)
nn.fit(X_train, y_train)
y_pred_nn = nn.predict(X_test)

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred_nn)).plot()
plt.title("Neural Network Confusion Matrix")
savefig("outputs/mp2_nn_confusion.png")

In [ ]:
# --- Random Forest ---
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred_rf)).plot()
plt.title("Random Forest Confusion Matrix")
savefig("outputs/mp2_rf_confusion.png")

In [ ]:
# --- XGBoost (if available) ---
if HAS_XGB:
    xgb = XGBClassifier(eval_metric="logloss", random_state=42)
    xgb.fit(X_train, y_train)
    y_pred_xgb = xgb.predict(X_test)

    ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred_xgb)).plot()
    plt.title("XGBoost Confusion Matrix")
    savefig("outputs/mp2_xgb_confusion.png")

In [ ]:
print("Mini Project 2 complete. Outputs saved in /outputs/")